In [556]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from ipywidgets import IntProgress
from sklearn.metrics import classification_report

import numpy as np

tf.enable_v2_behavior()

In [557]:
def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def tf_gaussian_mech(v, sensitivity, epsilon, delta):
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

In [558]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [559]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [560]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(28, kernel_size=(3,3), input_shape=input_shape),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),  
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])
model.summary()

models = [model]

Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 26, 26, 28)        280       
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 13, 13, 28)        0         
_________________________________________________________________
flatten_53 (Flatten)         (None, 4732)              0         
_________________________________________________________________
dense_106 (Dense)            (None, 128)               605824    
_________________________________________________________________
dropout_53 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_107 (Dense)            (None, 10)                1290      
Total params: 607,394
Trainable params: 607,394
Non-trainable params: 0
_______________________________________________

In [561]:
def tf_gaussian_mech(v, sensitivity, epsilon, delta):
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def tf_l2_clip(v, b):
    norm = tf.norm(v)
    return tf.cond(norm > b, lambda: b * (v / norm), lambda: v)



class DPOptimizer(tf.keras.optimizers.Optimizer):
    def __init__(self, b=3.0, learning_rate=0.01, name="DPOptimizer", **kwargs):
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", learning_rate)
        
        self.b = b
    
    def _create_slots(self, var_list):
        pass


    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
        }

    
class EpsilonDeltaDPGradientDescent(DPOptimizer):
    def __init__(self, epsilon, delta, b=3.0, learning_rate=0.01, name="EpsilonDeltaDPGradientDescent", **kwargs):
        DPOptimizer.__init__(self, b=b, learning_rate=learning_rate, name=name, **kwargs)        
        self.epsilon = epsilon
        self.delta = delta

    @tf.function
    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        
        clipped_grad = tf_l2_clip(grad, self.b)
        print(clipped_grad.shape)
        clipped_grad = tf.keras.backend.mean(tf_l2_clip(grad, self.b), axis=0)
        print(clipped_grad.shape)
        print()
        new_var_m = var - tf_gaussian_mech(clipped_grad, self.b/len(x_train), self.epsilon, self.delta) * lr_t
        #new_var_m = var - grad * lr_t
        
        new_var = new_var_m
        var.assign(new_var)

    
# class RenyiDPGradientDescent(tf.keras.optimizers.Optimizer):
#     def __init__(self, alpha, epsilon_bar, b=3.0, learning_rate=0.01, name="RenyiDPGradientDescent", **kwargs):
#         super().__init__(name, **kwargs)
#         self._set_hyper("learning_rate", learning_rate)
        
#         self.epsilon_bar = epsilon_bar
#         self.alpha = alpha
        
        
    
    

In [562]:
es = callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
optimizer = EpsilonDeltaDPGradientDescent(10, 1e-5, b=1, learning_rate=1)
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(x=x_train,y=y_train, epochs=5, callbacks=[es], batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
(3, 3, 1, 28)
(3, 1, 28)

(28,)
()

(4732, 128)
(128,)

(128,)
()

(128, 10)
(10,)

(10,)
()

938/938 [==============================] - 18s 19ms/step - loss: 1.7377 - accuracy: 0.4106 - val_loss: 1.3660 - val_accuracy: 0.5723
Epoch 2/5
938/938 [==============================] - 17s 18ms/step - loss: 1.5655 - accuracy: 0.4775 - val_loss: 1.5477 - val_accuracy: 0.4619
Epoch 3/5
938/938 [==============================] - 17s 18ms/step - loss: 1.5473 - accuracy: 0.4809 - val_loss: 1.3713 - val_accuracy: 0.5492
Epoch 4/5
938/938 [==============================] - 17s 18ms/step - loss: 1.5229 - accuracy: 0.4920 - val_loss: 1.2782 - val_accuracy: 0.5914
Epoch 5/5
938/938 [==============================] - 17s 19ms/step - loss: 1.5088 - accuracy: 0.4959 - val_loss: 1.2568 - val_accuracy: 0.6106


In [452]:
tf.random.normal([4], 0, 1, tf.float32)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 1.3733474 ,  0.95686656,  0.4819915 , -0.08288915], dtype=float32)>

In [344]:
t = tf.constant([1.0, 1.0, 1.0, 1.0])
t

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>

In [345]:
t_g = tf_gaussian_mech(t, 3.0/len(x_train), 0.1, 1e-5)
t_g

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1.0018165 , 0.9987338 , 0.99778914, 1.0040578 ], dtype=float32)>

In [346]:
norm = tf.norm(t)
norm.numpy()

2.0